<a href="https://colab.research.google.com/github/CamiloVga/Curso-IA-Para-Ciencia-de-Datos/blob/main/Script_Sesi%C3%B3n_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IA para la Ciencia de Datos
## Universidad de los Andes

**Profesor:** Camilo Vega - AI/ML Engineer  
**LinkedIn:** https://www.linkedin.com/in/camilo-vega-169084b1/

---

## Guía: Fine-tuning y RAG (Retrieval-Augmented Generation)

Este notebook presenta **3 implementaciones prácticas**:

1. **Fine-tuning Básico** - Análisis de sentimientos con tweets
2. **RAG Simple** - Búsqueda básica en documentos
3. **RAG Base de Datos** - Sistema especializado con embeddings

### Requisitos
- **GPU:** Tesla T4 mínimo (Colab gratuito)
- **APIs:** Hugging Face token
- **Datos:** Tweets y dataset de ventas

## Configuración APIs
- **Hugging Face:**
  1. [Crear token](https://huggingface.co/settings/tokens)
  2. En Colab: 🔑 Secrets → Agregar `HF_TOKEN` → Pegar tu token

Cada sección es **independiente** y puede ejecutarse por separado.


#Fine Tuning

In [ ]:
# 1. Instalaciones
!pip install torch transformers datasets scikit-learn matplotlib seaborn pandas huggingface_hub -q

# 2. Imports y configuración
from google.colab import userdata
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, classification_report, roc_auc_score
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
from huggingface_hub import login

# Configuración
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "bert-base-multilingual-cased"  # Modelo multilingüe para inglés

# Autenticación
hf_token = userdata.get('HF_TOKEN')
login(token=hf_token)
print(f"Dispositivo: {device}")
print(f"Modelo: {model_name} (multilingüe para textos en inglés)")

# 3. Cargar datos
print("Cargando dataset de tweets...")
dataset = load_dataset("mteb/tweet_sentiment_extraction", split="train")

textos = dataset["text"]
etiquetas = dataset["label"]  # 0=negative, 1=neutral, 2=positive

print(f"Dataset cargado: {len(textos)} muestras")
print(f"Distribución: {pd.Series(etiquetas).value_counts().sort_index().to_dict()}")

# 4. Filtrar y balancear
df = pd.DataFrame({'texto': textos, 'etiqueta': etiquetas})

# Tomar 5000 ejemplos balanceados
df_sample = df.groupby('etiqueta').apply(
    lambda x: x.sample(min(1667, len(x)), random_state=42)
).reset_index(drop=True).sample(frac=1, random_state=42)

textos_final = df_sample['texto'].tolist()
etiquetas_final = df_sample['etiqueta'].tolist()

print(f"Datos balanceados: {len(textos_final)} muestras")
print(f"Distribución final: {pd.Series(etiquetas_final).value_counts().sort_index().to_dict()}")

# 5. Split train/test
X_train, X_test, y_train, y_test = train_test_split(
    textos_final, etiquetas_final, test_size=0.2, random_state=42, stratify=etiquetas_final
)

# 6. Modelo y tokenizador
print("Cargando modelo...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3).to(device)

# 7. Tokenización
train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=128, return_tensors="pt")
test_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=128, return_tensors="pt")

# 8. Dataset clase
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(train_encodings, y_train)
test_dataset = SentimentDataset(test_encodings, y_test)

# 9. Métricas (DEFINIR ANTES DE USAR)
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    return {'accuracy': accuracy, 'f1': f1, 'precision': precision, 'recall': recall}

# 10. Configuración de entrenamiento (DEFINIR ANTES DE USAR)
training_args = TrainingArguments(
    output_dir='./modelo',
    num_train_epochs=5,  # Reducir épocas para evitar divergencia
    per_device_train_batch_size=32,  # Batch más grande para estabilidad
    per_device_eval_batch_size=64,
    learning_rate=1e-5,  # Learning rate mucho más bajo
    weight_decay=0.1,    # Más regularización
    warmup_steps=200,
    eval_strategy="epoch",
    logging_steps=50,
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",  # Optimizar por accuracy
    greater_is_better=True,
    report_to=[],
    dataloader_drop_last=True,
    gradient_checkpointing=True,  # Ahorrar memoria
    fp16=True  # Precisión mixta para estabilidad
)

# 11. Entrenar modelo estándar (AHORA CON VARIABLES DEFINIDAS)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

print("\n" + "="*50)
print("ENTRENANDO...")
print("="*50)

# Entrenar y capturar historial
history = trainer.train()

# Extraer métricas del historial de logs
train_losses = []
eval_losses = []
eval_accuracies = []

for log in trainer.state.log_history:
    if 'train_loss' in log:
        train_losses.append(log['train_loss'])
    if 'eval_loss' in log:
        eval_losses.append(log['eval_loss'])
    if 'eval_accuracy' in log:
        eval_accuracies.append(log['eval_accuracy'])

print(f"Entrenamiento completado. Logs capturados: {len(eval_losses)} evaluaciones")

# 12. Gráficas usando historial de logs
if eval_losses:
    plt.figure(figsize=(15, 4))
    epochs = range(1, len(eval_losses) + 1)

    # Pérdida de evaluación
    plt.subplot(1, 3, 1)
    plt.plot(epochs, eval_losses, 'r-o', linewidth=2, markersize=6)
    plt.title('Pérdida de Evaluación')
    plt.xlabel('Época')
    plt.ylabel('Loss')
    plt.grid(True, alpha=0.3)

    # Accuracy
    plt.subplot(1, 3, 2)
    if eval_accuracies and len(eval_accuracies) == len(eval_losses):
        plt.plot(epochs, eval_accuracies, 'g-o', linewidth=2, markersize=6)
    plt.title('Accuracy')
    plt.xlabel('Época')
    plt.ylabel('Accuracy')
    plt.grid(True, alpha=0.3)

    # Train loss si está disponible
    plt.subplot(1, 3, 3)
    if train_losses:
        # Si hay múltiples train losses por época, promediarlos
        if len(train_losses) > len(epochs):
            steps_per_epoch = len(train_losses) // len(epochs)
            train_by_epoch = []
            for i in range(len(epochs)):
                start = i * steps_per_epoch
                end = min((i + 1) * steps_per_epoch, len(train_losses))
                if start < len(train_losses):
                    epoch_avg = sum(train_losses[start:end]) / len(train_losses[start:end])
                    train_by_epoch.append(epoch_avg)

            if len(train_by_epoch) == len(epochs):
                plt.plot(epochs, train_by_epoch, 'b-s', linewidth=2, alpha=0.7, label='Train Loss')
                plt.plot(epochs, eval_losses, 'r-o', linewidth=2, label='Eval Loss')
                plt.legend()
        else:
            plt.plot(epochs, eval_losses, 'r-o', linewidth=2)
    else:
        plt.plot(epochs, eval_losses, 'r-o', linewidth=2)

    plt.title('Curvas de Pérdida')
    plt.xlabel('Época')
    plt.ylabel('Loss')
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    print(f"\nRESUMEN ENTRENAMIENTO:")
    if len(eval_losses) > 1:
        print(f"Loss inicial: {eval_losses[0]:.4f}")
        print(f"Loss final: {eval_losses[-1]:.4f}")
        print(f"Mejora: {eval_losses[0] - eval_losses[-1]:.4f}")

    if eval_accuracies and len(eval_accuracies) > 1:
        print(f"Accuracy inicial: {eval_accuracies[0]:.4f}")
        print(f"Accuracy final: {eval_accuracies[-1]:.4f}")
        print(f"Mejora: {eval_accuracies[-1] - eval_accuracies[0]:.4f}")

else:
    print("No se encontraron métricas de evaluación en el historial")

# 13. Evaluación final
predictions = trainer.predict(test_dataset)
y_pred = np.argmax(predictions.predictions, axis=1)
y_pred_proba = torch.softmax(torch.tensor(predictions.predictions), dim=1).numpy()

print("\n" + "="*50)
print("RESULTADOS")
print("="*50)

# Verificar que las dimensiones coincidan
print(f"Muestras de prueba: {len(y_test)}")
print(f"Predicciones: {len(y_pred)}")

# Labels names
labels_names = ['Negativo', 'Neutral', 'Positivo']

# Solo evaluar si las dimensiones coinciden
if len(y_test) == len(y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    print(f"ACCURACY: {accuracy:.4f}")

    # Matriz de confusión
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=labels_names, yticklabels=labels_names)
    plt.title('Matriz de Confusión')
    plt.show()

    # ROC-AUC
    y_true_bin = label_binarize(y_test, classes=[0, 1, 2])
    roc_auc = roc_auc_score(y_true_bin, y_pred_proba, multi_class='ovr', average='weighted')
    print(f"ROC-AUC: {roc_auc:.4f}")

    # Classification report
    print("\nREPORTE:")
    print(classification_report(y_test, y_pred, target_names=labels_names))
else:
    print(f"Error: Inconsistencia en dimensiones - y_test: {len(y_test)}, y_pred: {len(y_pred)}")
    print("Evaluando solo con las primeras muestras que coincidan...")

    min_len = min(len(y_test), len(y_pred))
    y_test_truncated = y_test[:min_len]
    y_pred_truncated = y_pred[:min_len]

    accuracy = accuracy_score(y_test_truncated, y_pred_truncated)
    print(f"ACCURACY (truncado): {accuracy:.4f}")

    # Matriz de confusión truncada
    cm = confusion_matrix(y_test_truncated, y_pred_truncated)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=labels_names, yticklabels=labels_names)
    plt.title('Matriz de Confusión')
    plt.show()

# 14. Guardar modelo
trainer.save_model("./sentiment_model")
tokenizer.save_pretrained("./sentiment_model")
print("Modelo guardado en ./sentiment_model")

# 15. Inferencia individual simple
from transformers import pipeline

# Cargar el modelo entrenado
sentiment_pipeline = pipeline("text-classification", model="./sentiment_model", tokenizer="./sentiment_model")

# Ejemplo de uso: cambiar el texto aquí para probar
texto_prueba = "I love this amazing product!"

resultado = sentiment_pipeline(texto_prueba)[0]
label_idx = int(resultado['label'].split('_')[1])
sentimientos = ['Negativo', 'Neutral', 'Positivo']

print("\n" + "="*40)
print("INFERENCIA INDIVIDUAL")
print("="*40)
print(f"Texto: '{texto_prueba}'")
print(f"Sentimiento: {sentimientos[label_idx]}")
print(f"Confianza: {resultado['score']:.3f}")


#RAG Documentos

In [ ]:
# RAG SIMPLE CON GROQ API
# Sistema de búsqueda en documentos usando TF-IDF y Groq para generación

# Instalación de dependencias
!pip install groq scikit-learn PyPDF2 python-docx pandas openpyxl -q

import os
import pandas as pd
import numpy as np
from groq import Groq
from google.colab import userdata
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from PyPDF2 import PdfReader
from docx import Document

# Configuración del sistema
GROQ_API_KEY = userdata.get('GROQ_KEY')
client = Groq(api_key=GROQ_API_KEY)
folder_path = '/content/carpeta_rag'

# Variables globales del sistema
vectorizer = TfidfVectorizer(max_features=1000, stop_words=None)
chunks = []
vectors = None

def load_documents(folder_path):
    """Carga y procesa documentos de múltiples formatos"""
    text = ''
    os.makedirs(folder_path, exist_ok=True)

    for filename in os.listdir(folder_path):
        filepath = os.path.join(folder_path, filename)

        try:
            if filename.endswith('.txt'):
                with open(filepath, 'r', encoding='utf-8') as f:
                    text += f.read() + '\n'

            elif filename.endswith('.pdf'):
                reader = PdfReader(filepath)
                for page in reader.pages:
                    text += page.extract_text() + '\n'

            elif filename.endswith('.docx'):
                doc = Document(filepath)
                for paragraph in doc.paragraphs:
                    text += paragraph.text + '\n'

            elif filename.endswith('.csv'):
                df = pd.read_csv(filepath)
                text += df.to_string() + '\n'

            elif filename.endswith('.xlsx'):
                df = pd.read_excel(filepath)
                text += df.to_string() + '\n'
        except:
            continue

    return text

def create_chunks(text, chunk_size=500):
    """Divide el texto en fragmentos de tamaño manejable"""
    words = text.split()
    chunks = []

    for i in range(0, len(words), chunk_size):
        chunk = ' '.join(words[i:i + chunk_size])
        if len(chunk.strip()) > 100:
            chunks.append(chunk.strip())

    return chunks

def search_relevant_chunks(query, top_k=3):
    """Encuentra fragmentos más relevantes usando similitud coseno"""
    query_vector = vectorizer.transform([query])
    similarities = cosine_similarity(query_vector, vectors)[0]

    # Obtener índices ordenados por relevancia
    top_indices = np.argsort(similarities)[-top_k:][::-1]

    relevant_chunks = []
    for idx in top_indices:
        if similarities[idx] > 0.1:  # Umbral mínimo de relevancia
            relevant_chunks.append(chunks[idx])

    return relevant_chunks

def generate_answer(query, context):
    """Genera respuesta usando Groq API con el contexto encontrado"""
    if not context:
        return "No encontré información relevante en los documentos."

    prompt = f"""Basándote en esta información:

{context}

Pregunta: {query}

Instrucciones:
- Responde solo con información del contexto
- Si no hay información suficiente, di que no puedes responder
- Responde en español de forma clara y concisa"""

    try:
        response = client.chat.completions.create(
            model="openai/gpt-oss-20b",
            messages=[
                {"role": "user", "content": prompt}
            ],
            max_tokens=400,
            temperature=0.7
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error generando respuesta: {e}"

def inicializar_rag():
    """Inicializa el sistema RAG cargando documentos y creando índices"""
    global chunks, vectors, vectorizer

    print("Cargando documentos...")
    documents = load_documents(folder_path)

    if not documents.strip():
        print("No se encontraron documentos en la carpeta.")
        return False

    print("Creando fragmentos de texto...")
    chunks = create_chunks(documents)

    print("Generando índice de búsqueda...")
    vectors = vectorizer.fit_transform(chunks)

    print(f"Sistema listo: {len(chunks)} fragmentos procesados.")
    return True

# Inicialización del sistema
inicializar_rag()

# BLOQUE DE INFERENCIA - Ejecutar por separado
query = "¿De qué tratan estos documentos?"  # Cambia tu pregunta aquí
context = '\n\n'.join(search_relevant_chunks(query))
respuesta = client.chat.completions.create(model="openai/gpt-oss-20b", messages=[{"role": "user", "content": f"Contexto: {context}\nPregunta: {query}\nResponde solo con información del contexto en español."}], max_tokens=400).choices[0].message.content
print(query)
print(respuesta)

#Rag Base de Datos


In [ ]:
# RAG BASE DE DATOS CON GROQ API
# Sistema de consultas inteligentes sobre bases de datos usando embeddings y LLM
# Convierte preguntas en lenguaje natural a SQL ejecutable

"""
ARQUITECTURA DEL SISTEMA:

1. Carga de datos: CSV → SQLite en memoria
2. Análisis de esquema: Detecta tipos y genera embeddings de columnas
3. Búsqueda semántica: Encuentra columnas relevantes para la consulta
4. Generación SQL: LLM crea consulta SQL basada en esquema y pregunta
5. Ejecución: Consulta todas las observaciones sin límites
6. Respuesta: LLM genera respuesta natural con estadísticas precisas
"""

# Instalación de dependencias
!pip install groq sentence-transformers pandas numpy scikit-learn -q

import pandas as pd
import numpy as np
import sqlite3
import os
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from groq import Groq
from google.colab import userdata

# Configuración del sistema
GROQ_API_KEY = userdata.get('GROQ_KEY')
client = Groq(api_key=GROQ_API_KEY)

# Modelo centralizado para ambos componentes del sistema
GROQ_MODEL = "openai/gpt-oss-20b"

# Configuración de paths y variables globales
folder_path = '/content/carpeta_rag'
model = SentenceTransformer('all-MiniLM-L6-v2')
conn = None
table_name = 'data'
column_embeddings = {}
column_info = {}
schema_description = ""

def load_database(folder_path):
    """Carga CSV desde carpeta y crea base de datos en memoria"""
    global conn, column_embeddings, column_info, schema_description

    os.makedirs(folder_path, exist_ok=True)

    csv_file = None
    try:
        files = os.listdir(folder_path)
        print(f"Archivos en carpeta: {files}")

        # Buscar primer archivo CSV
        for filename in files:
            if filename.endswith('.csv'):
                csv_file = os.path.join(folder_path, filename)
                break

        if not csv_file:
            print("Error: No se encontró ningún archivo CSV en la carpeta.")
            return False

        print(f"Cargando {csv_file}...")
        df = pd.read_csv(csv_file)

        # Crear conexión SQLite en memoria
        if conn:
            conn.close()
        conn = sqlite3.connect(':memory:', check_same_thread=False)
        df.to_sql(table_name, conn, index=False, if_exists='replace')

        # Generar embeddings y descripción del esquema
        column_embeddings.clear()
        column_info.clear()
        schema_parts = []

        for col in df.columns:
            # Detectar tipo de columna automáticamente
            if df[col].dtype in ['int64', 'float64']:
                col_type = 'NUMERIC'
                sample_values = f"rango: {df[col].min()} - {df[col].max()}"
                desc = f"{col} (numérica): {sample_values}"
            elif df[col].nunique() <= 20:
                col_type = 'CATEGORICAL'
                unique_vals = list(df[col].unique())
                sample_values = f"valores: {', '.join(map(str, unique_vals[:5]))}"
                desc = f"{col} (categórica): {sample_values}"
            else:
                col_type = 'TEXT'
                sample_values = f"texto con {df[col].nunique()} valores únicos"
                desc = f"{col} (texto): {sample_values}"

            column_info[col] = {
                'type': col_type,
                'sample_values': sample_values,
                'description': desc
            }

            # Generar embedding para búsqueda semántica
            column_embeddings[col] = model.encode([desc])[0]
            schema_parts.append(f"- {col} ({col_type}): {sample_values}")

        schema_description = f"""
Tabla: {table_name}
Total de registros: {len(df)}
Columnas disponibles:
""" + "\n".join(schema_parts)

        print(f"Base de datos cargada: {df.shape[0]} filas, {df.shape[1]} columnas")
        print("Esquema detectado:")
        print(schema_description)
        return True

    except Exception as e:
        print(f"Error cargando datos: {e}")
        return False

def find_relevant_columns(query, top_k=5):
    """Encuentra las columnas más relevantes usando búsqueda semántica"""
    query_emb = model.encode([query])[0]
    scores = []
    for col, col_emb in column_embeddings.items():
        sim = cosine_similarity([query_emb], [col_emb])[0][0]
        scores.append((col, sim, column_info[col]))
    return sorted(scores, key=lambda x: x[1], reverse=True)[:top_k]

def generate_sql_with_llm(query):
    """Genera consulta SQL usando LLM con contexto del esquema"""
    relevant_cols = find_relevant_columns(query)

    # Construir contexto de columnas relevantes
    cols_context = "\nColumnas más relevantes para la consulta:\n"
    for col, score, info in relevant_cols:
        cols_context += f"- {col} ({info['type']}): {info['sample_values']}\n"

    prompt = f"""Eres un experto en SQL. Genera una consulta SQL para responder la pregunta del usuario.

ESQUEMA DE LA BASE DE DATOS:
{schema_description}

{cols_context}

PREGUNTA DEL USUARIO: {query}

INSTRUCCIONES:
1. Usa el nombre de tabla: {table_name}
2. NO uses LIMIT - necesitamos todos los datos
3. Si necesitas agrupar, usa GROUP BY apropiadamente
4. Si necesitas ordenar, usa ORDER BY
5. Para consultas numéricas usa SUM, AVG, COUNT, etc.
6. Responde SOLO con la consulta SQL, sin explicaciones

CONSULTA SQL:"""

    try:
        response = client.chat.completions.create(
            model=GROQ_MODEL,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=200,
            temperature=0.1
        )
        sql_generated = response.choices[0].message.content.strip()

        # Limpiar la respuesta
        sql_generated = sql_generated.replace('```sql', '').replace('```', '').strip()

        return sql_generated
    except Exception as e:
        print(f"Error generando SQL: {e}")
        # Fallback simple
        relevant_col_names = [col for col, _, _ in relevant_cols[:3]]
        return f"SELECT {', '.join(relevant_col_names)} FROM {table_name}"

def execute_query(sql):
    """Ejecuta la consulta SQL en la base de datos"""
    global conn
    if conn is None:
        return "Error: Base de datos no inicializada", pd.DataFrame()

    try:
        print(f"Ejecutando SQL: {sql}")
        resultado = pd.read_sql_query(sql, conn)
        return sql, resultado
    except Exception as e:
        print(f"Error SQL: {e}")
        # Fallback: mostrar todas las columnas relevantes
        try:
            fallback_sql = f"SELECT * FROM {table_name}"
            resultado = pd.read_sql_query(fallback_sql, conn)
            return fallback_sql, resultado
        except Exception as e2:
            print(f"Error en consulta fallback: {e2}")
            return "Error", pd.DataFrame()

def generate_answer(query, sql_executed, data):
    """Genera respuesta natural usando LLM con análisis estadístico"""
    if data.empty:
        return "No se encontraron datos para esta consulta."

    # Preparar estadísticas automáticas
    stats_summary = f"Se procesaron {len(data)} registros totales.\n"

    # Agregar estadísticas de columnas numéricas
    numeric_cols = data.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        stats_summary += "\nEstadísticas numéricas:\n"
        for col in numeric_cols[:3]:
            stats_summary += f"- {col}: total={data[col].sum():.2f}, promedio={data[col].mean():.2f}\n"

    # Preparar muestra de datos
    data_sample = data.head(15).to_string(index=False) if len(data) > 15 else data.to_string(index=False)
    if len(data) > 15:
        data_sample += f"\n... (mostrando 15 de {len(data)} registros)"

    prompt = f"""Analiza estos resultados de base de datos y responde la pregunta del usuario de forma clara y profesional.

PREGUNTA ORIGINAL: {query}
SQL EJECUTADO: {sql_executed}

ESTADÍSTICAS:
{stats_summary}

DATOS OBTENIDOS:
{data_sample}

INSTRUCCIONES:
- Responde basándote en los {len(data)} registros procesados
- Da números exactos y estadísticas precisas
- Responde en español de forma profesional y directa
- Si hay rankings, menciona los elementos más importantes
- No repitas el SQL ni la pregunta

RESPUESTA:"""

    try:
        response = client.chat.completions.create(
            model=GROQ_MODEL,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=300,
            temperature=0.2
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error generando respuesta: {e}"

def query_database(user_query):
    """Función principal para procesar consultas de usuario"""
    print(f"\n🔍 Procesando consulta: {user_query}")

    # Pipeline completo del RAG
    sql = generate_sql_with_llm(user_query)
    sql_executed, data = execute_query(sql)
    respuesta = generate_answer(user_query, sql_executed, data)

    print(f"📊 SQL ejecutado: {sql_executed}")
    print(f"💡 Respuesta: {respuesta}")

    return respuesta, data

def inicializar_rag():
    """Inicializa el sistema RAG completo"""
    print("🚀 Iniciando sistema RAG...")
    if load_database(folder_path):
        print("✅ Sistema RAG listo para consultas.")
        return True
    else:
        print("❌ Error iniciando RAG.")
        return False

# Inicialización del sistema
inicializar_rag()

# Ejemplo de uso
query = "¿cuál es la ciudad con mayores ventas?"
sql = generate_sql_with_llm(query)
sql_executed, data = execute_query(sql)
respuesta = generate_answer(query, sql_executed, data)
print(f"Pregunta: {query}")
print(f"SQL: {sql_executed}")
print(f"Respuesta: {respuesta}")

In [ ]:
# BLOQUE DE INFERENCIA - ejemplo
query = "¿Cuáles son los colores que más venden?"
sql = generate_sql_with_llm(query)
sql_executed, data = execute_query(sql)
respuesta = generate_answer(query, sql_executed, data)
print(f"Pregunta: {query}")
print(f"SQL: {sql_executed}")
print(f"Respuesta: {respuesta}")

In [ ]:
# BLOQUE DE INFERENCIA - ejemplo
query = "¿cuál es el producto que más genera ingresos??"
sql = generate_sql_with_llm(query)
sql_executed, data = execute_query(sql)
respuesta = generate_answer(query, sql_executed, data)
print(f"Pregunta: {query}")
print(f"SQL: {sql_executed}")
print(f"Respuesta: {respuesta}")

In [ ]:
# BLOQUE DE INFERENCIA - ejemplo
query = "¿cuál es la talla que más se vende?"
sql = generate_sql_with_llm(query)
sql_executed, data = execute_query(sql)
respuesta = generate_answer(query, sql_executed, data)
print(f"Pregunta: {query}")
print(f"SQL: {sql_executed}")
print(f"Respuesta: {respuesta}")

In [ ]:
# BLOQUE DE INFERENCIA - ejemplo
query = "¿Cuánto suman las transacciones de la talla M?"
sql = generate_sql_with_llm(query)
sql_executed, data = execute_query(sql)
respuesta = generate_answer(query, sql_executed, data)
print(f"Pregunta: {query}")
print(f"SQL: {sql_executed}")
print(f"Respuesta: {respuesta}")

In [ ]:
# BLOQUE DE INFERENCIA - ejemplo
query = "¿cuál es el método de pago más importante"
sql = generate_sql_with_llm(query)
sql_executed, data = execute_query(sql)
respuesta = generate_answer(query, sql_executed, data)
print(f"Pregunta: {query}")
print(f"SQL: {sql_executed}")
print(f"Respuesta: {respuesta}")